### Step1: 调用LLM理解用户意图

In [ ]:
import { config } from "npm:dotenv";
config();

In [30]:
// 1. 配置 DeepSeek API
const DEEPSEEK_API_KEY = process.parsed.DEEPSEEK_API_KEY;
const DEEPSEEK_API_BASE = "https://api.deepseek.com";

In [31]:
// 1. 手动构建 prompt
const userInput = "上海今天天气怎么样？";

const prompt = `
    你是一个智能助手，判断用户是否在查询天气。
    如果是，提取城市名称。

    用户问题：${userInput}

    请返回 JSON 格式：
    {"action": "query_weather", "city": "城市名称"}
`

In [32]:
// 3. 调用 DeepSeek API
// reference:
const response = await fetch(`${DEEPSEEK_API_BASE}/chat/completions`, {
  method: "POST",
  headers: {
    "Content-Type": "application/json",
    "Authorization": `Bearer ${DEEPSEEK_API_KEY}`
  },
  body: JSON.stringify({
    model: "deepseek-chat",
    messages: [
      { role: "system", content: "你是一个助手" },
      { role: "user", content: prompt }
    ]
  })
});

const data = await response.json();

In [33]:
// 4. 解析 LLM 返回的结果
const llmOutput = data.choices[0].message.content;
const parsed = JSON.parse(llmOutput);
const city = parsed.city; // 提取出"上海"
console.log(`提取的城市名称: ${city}`);

提取的城市名称: 上海


### Step2: 调用天气API

In [34]:
// 5. 调用天气 API
const WEATHER_API_KEY = Deno.env.get("WEATHER_API_KEY");
const WEATHER_BASE_URL = `http://api.weatherapi.com/v1/current.json?key=${WEATHER_API_KEY}&q=${city}`;
const weatherResponse = await fetch(WEATHER_BASE_URL);
const weatherData = await weatherResponse.json();

// 6. 提取天气信息
const temperature = weatherData.current.temp_c;
const condition = weatherData.current.condition.text;

### Step 3：再次调用 LLM 生成自然语言回答

In [35]:
// 7. 构建新的 prompt，让 LLM 把数据转成自然语言
const finalPrompt = `
    根据以下天气数据，生成一个友好的回答：
    城市：${city}
    温度：${temperature}°C
    天气：${condition}

    用户问题：${userInput}
`;

const finalResponse = await fetch(`${DEEPSEEK_API_BASE}/chat/completions`, {
  method: "POST",
  headers: {
    "Content-Type": "application/json",
    "Authorization": `Bearer ${DEEPSEEK_API_KEY}`
  },
  body: JSON.stringify({
    model: "deepseek-chat",
    messages: [
      { role: "system", content: "你是一个助手" },
      { role: "user", content: finalPrompt }
    ]
  })
});

const finalData = await finalResponse.json();
const answer = finalData.choices[0].message.content;
console.log(answer); // "上海今天天气不错哦！ 当前温度是25°C，天气晴朗，适合外出游玩！"

上海今天天气很不错哦！🌞 目前气温19.4°C，天气晴朗，适合外出活动。记得享受这美好的天气呀！


**面临问题1: 如果要换成 OpenAI 会怎样？**

噩梦开始！需要重写所有代码：不同的API调用方式，不同的响应格式...

**面临问题2: 如何记录对话历史？**
自己实现

In [36]:
interface Message {
  role: string;
  content: string;
}

const conversationHistory: Message[] = [];

async function chat(userInput: string): Promise<string> {
  // 手动管理历史
  conversationHistory.push({ role: "user", content: userInput });
  
  // 每次调用都要传入完整历史
  const response = await fetch(`${DEEPSEEK_API_BASE}/chat/completions`, {
    method: "POST",
    headers: {
      "Content-Type": "application/json",
      "Authorization": `Bearer ${DEEPSEEK_API_KEY}`
    },
    body: JSON.stringify({
      model: "deepseek-chat",
      messages: conversationHistory
    })
  });
  
  const data = await response.json();
  const assistantReply = data.choices[0].message.content;
  conversationHistory.push({ role: "assistant", content: assistantReply });
  
  // 还要自己管理历史长度，防止超过 token 限制
  if (conversationHistory.length > 20) {
    conversationHistory.splice(0, conversationHistory.length - 20);
  }
  
  return assistantReply;
}